# Sentence Transformers

In questo notebook faccio fine-tuning di un [Sentence Transformer](https://arxiv.org/abs/1908.10084) per l'italiano!

<br>

![cover](https://huggingface.co/blog/assets/95_training_st_models/training_process.png)

<br>

> Questo notebook è basato sul blogpost [How to train sentence-transformers](https://huggingface.co/blog/how-to-train-sentence-transformers).

### Cos'è un Sentence-Transformers?

Un Sentence-Transformers mappa un testo in un embedding di dimensione fissa che ne rappresenta il significato.

### Come funziona?

Il Sentence-Transformers non è altro che un modello Transformer, di quelli che possiamo scaricare liberamente da HuggingFace al quale viene aggiunto un livello di pooling per dare una dimensione fissa ai vettori in uscita, di solito si usa il *mean pooling*.

Per alcuni task (es. clustering) a volte viene aggiunto anche un livello di *Normalization* alla fine ma non è il focus di questo notebook.

## Come fare fine-tuning di un Sentence Tansformers

Per addestrare o fare fine tuning di un sentence transformer le prime due cose da conoscere sono:
* la disponibilità dei dati e la struttura del dataset di addestramento
* le loss function e la loro relazione con la struttura del dataset

In questo notebook verrà creato un sentence-transformers per l'italiano che alla fine è stato caricato su [Huggingface](https://huggingface.co).

In [ ]:
!pip install -U sentence-transformers

## Scelta del modello

La prima cosa da fare è sceglieri il modello adatto al proprio obiettivo, il nostro è costruire un modello per l'italiano, possiamo farlo in due modi:
1. Prendere un Transformers monolingua già addestrato per il [fill-mask](https://huggingface.co/tasks/fill-mask)
2. Prendere un multilingua

Ho optato per la prima scelta anche se i checkpoint HuggingFace per il fill-mask in italiano sono molto pochi, nelle prossime celle andremo a costruire il Sentence-Transformers sfruttando [l'omonima libreria](https://www.sbert.net/)

In [ ]:
from sentence_transformers import SentenceTransformer, models

model_checkpoint = "dbmdz/bert-base-italian-uncased"

In [ ]:
word_embedding_model = models.Transformer(model_name_or_path=model_checkpoint)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
#normalize_model = models.Normalize()

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
model

#### Perchè non usiamo direttamente un modello come BERT?

* Usare un sentence transofrmers al posto del transformers di base abbassa i **tempi di elaborazione**. Ad esempio [fare ricerca su 10000 frasi può richedere circa 65 ore con BERT e 5 secondi con Sentence Transformers](https://arxiv.org/abs/1908.10084).

* I Transformers **creano rappresentazioni scarse**, a livello (o anche sotto) di algoritmi più datati come [GloVe](https://nlp.stanford.edu/projects/glove/).

<br>

![sbert](https://www.sbert.net/_static/logo.png)

<br>

## Prepare dataset

I Sentence Transformers vengono utilizzati per calcolare la similarità tra frasi, quindi il dataset di train deve essere strutturato in modo da far capire al modello quali sono le frasi simili e quanto lo sono. Esistono diverse strutture di dataset per questo task.

1. Le coppie di frasi hanno una label (*int* o *float*) che indica il grado di similarità
2. Coppie di frasi simili senza etichetta. Ad esempio: testo->riassunto, domande uguali formulate in modo diverso, testo->traduzione, ...
3. La frase ha una label intera che ne rappresenta il tipo. Il dataset viene formato da triplette *[anchor, positive, negative]* dove le prime due hanno la stessa label.
4. Triplette *[anchor, positive, negative]* ma senza label

In questo caso ho usato il dataset del primo tipo utilizzando la traduzione in italiano di [STSbenchmark dataset](https://ixa2.si.ehu.eus/stswiki/index.php/STSbenchmark).

> N.B. I casi 1 e 3 sono i più semplici su cui lavorare, il più complesso è il 4.

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset


In [ ]:
dataset_train = load_dataset("stsb_multi_mt", name="it", split="train")
dataset_test = load_dataset("stsb_multi_mt", name="it", split="test")

In [ ]:
print("train size: ", dataset_train.shape, "\n", "test size: ", dataset_test.shape)

### Train



In [ ]:
dataset_train[:3]

Come possiamo vedere la similarity score và da 0 a 5, per il fine tuning del modello dobbiamo portarla tra 0 e 1.

Bisogna creare una struttura ([sentence1, sentence2], label) usando l'oggetto [InputExample](https://github.com/UKPLab/sentence-transformers/blob/master/sentence_transformers/readers/InputExample.py). Il risultato di questo passaggio viene usato come input del DataLoader.

In [ ]:
from sentence_transformers import InputExample

train_examples = []

for i in range(dataset_train.shape[0]):
  example = dataset_train[i]
  train_examples.append(InputExample(texts=[example['sentence1'], example['sentence2']], label = float(example['similarity_score'])/5.0))

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

### Evaluator

La libreria sentence_transformers mette a disposizione diversi costrutti per valutare le performance dei modelli, useremo [EmbeddingSimilarityEvaluator](https://www.sbert.net/docs/package_reference/evaluation.html#sentence_transformers.evaluation.EmbeddingSimilarityEvaluator) che valuta un modello in base alla somiglianza degli embeddings, calcolando la correlazione di Spearman e di Pearson rispetto alle gold standard labels. Le metriche disponibili sono la similarità del coseno, la distanza euclidea e la distanza di Manhattan. 

Lo score in output è la correlazione di Spearman.

In [ ]:
from sentence_transformers import evaluation

In [ ]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(dataset_test['sentence1'], dataset_test['sentence2'], [x/5.0 for x in dataset_test['similarity_score']],
                                                   main_similarity = evaluation.SimilarityFunction.COSINE, show_progress_bar=True, write_csv=True)

#### Losses

A seconda della struttura del dataset che stiamo usando bisogna utilizzare una loss. Nel nostro caso utilizzeremo la [CosineSimilarityLoss](https://www.sbert.net/docs/package_reference/losses.html#cosinesimilarityloss).

Lascio l'approfondimento delle losses al post originale, qui metto solo lo schema condiviso nello stesso.

> Come si vede dallo schema per la nostra tipologia di dataset potevamo scegliere diverse loss, la CosineSimilarityLoss lavora con la label float mentre le altre due con gli interi, da qui la scelta.

<br>

![losses](https://huggingface.co/blog/assets/95_training_st_models/datasets_table.png)

<br>

In [ ]:
from sentence_transformers import losses

In [ ]:
train_loss = losses.CosineSimilarityLoss(model = model)

## Fit the model

La parte più complicata è finita, ora non ci resta che far partire l'addestramento.
Impostiamo alcuni iperparametri, il più interessante è **warmup_step**. In pratica dopo un certo numero di passi (su molti forum consigliano il 10% del train) il learning rate viene ridotto linearmente fino allo 0.

In [ ]:
num_epochs = 10
evaluation_steps = 500

warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)], evaluator= evaluator, evaluation_steps = evaluation_steps,
          epochs=num_epochs, steps_per_epoch= 1500, warmup_steps=warmup_steps, save_best_model=True)

In [ ]:
# salva in locale
#model.save("my-awesome-sentence-transformer")

## Condividi sull'Hub

Per condividere i nostri modelli sull'Huggingface Hub c'è bisogno di un token, lo troviamo sul nostro profilo HuggingFace-->Settings--> Access Token.

> Non dimenticate di compilare una bella model card! :blush:

In [ ]:
#from huggingface_hub import login
#login('yourHFtoken')

In [ ]:
#model.save_to_hub('my-awesome-sentence-transformer', organization='yourUsername', train_datasets=[stsb_multi_mt])

## Utilizzo

Per utilizzare il nostro Sentence Transformer possiamo utilizzare sia l'omonima libreria che Transformsers di HuggingFace, io consiglio la prima opzione.

### Utilizzo con sentence-transformers

In [ ]:
#!pip install -U sentence-transformers

#from sentence_transformers import SentenceTransformer
#sentences = ["Una ragazza si acconcia i capelli.", "Una ragazza si sta spazzolando i capelli."]

#model = SentenceTransformer('yourUsername/my-awesome-sentence-transformer')
#embeddings = model.encode(sentences)
#print(embeddings)


### Utilizzo con tranformers

In [ ]:
"""
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['Una ragazza si acconcia i capelli.', 'Una ragazza si sta spazzolando i capelli.']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('yourUsername/my-awesome-sentence-transformer')
model = AutoModel.from_pretrained('yourUsername/my-awesome-sentence-transformer')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, mean pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)
"""

#### Altri riferimenti

* [Sentence embedding fine tuning for the french language](https://lajavaness.medium.com/sentence-embedding-fine-tuning-for-the-french-language-65e20b724e88)

* [Sentence Transformers and Embedding Evaluation](https://txt.cohere.com/sentence-transformers-embedding-evaluation/)

<br>

![huggingface](https://huggingface.co/front/thumbnails/v2-2.png)

<br>